Importing the libraries

In [1]:
import numpy as np
import pandas as pd

# Loading the dataset

In [2]:
train_df      = pd.read_csv('./train.csv')
test_df       = pd.read_csv('./test.csv')
submission_df = pd.read_csv('./submission.csv')

In [3]:
train_df

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
35990,35991,NaN,Zimbabwe,2020-05-11,36.0,4.0
35991,35992,NaN,Zimbabwe,2020-05-12,36.0,4.0
35992,35993,NaN,Zimbabwe,2020-05-13,37.0,4.0
35993,35994,NaN,Zimbabwe,2020-05-14,37.0,4.0


# Feature engineering

In [4]:
train_df.describe()

,Id,ConfirmedCases,Fatalities
count,35995.000000,35995.000000,35995.000000
mean,17998.000000,3683.508737,243.560217
std,10391.005806,18986.978708,1832.966999
min,1.000000,0.000000,0.000000
25%,8999.500000,0.000000,0.000000
50%,17998.000000,19.000000,0.000000
75%,26996.500000,543.000000,7.000000
max,35995.000000,345813.000000,33998.000000


In [5]:
train_df.shape

(35995, 6)

## Check for missing values

In [6]:
display(train_df.isnull().sum() / len(train_df) * 100)
display(test_df.isnull().sum() / len(test_df) * 100)

Id                 0.000000
Province_State    57.507987
Country_Region     0.000000
Date               0.000000
ConfirmedCases     0.000000
Fatalities         0.000000
dtype: float64

ForecastId         0.000000
Province_State    57.507987
Country_Region     0.000000
Date               0.000000
dtype: float64

## Check for data range in both datasets

In [7]:
print("The lowest date in the train dataset is {} and the highest {}".format(train_df['Date'].min(), train_df['Date'].max()))
print("The lowest date in the test dataset is {} and the highest {}".format(test_df['Date'].min(), test_df['Date'].max()))

The lowest date in the train dataset is 2020-01-22 and the highest 2020-05-15
The lowest date in the test dataset is 2020-04-02 and the highest 2020-05-14


## Renaming some features

In [8]:
train_df.rename(columns={'Province_State':'State','Country_Region':'Country'}, inplace=True)
test_df.rename(columns={'Province_State':'State','Country_Region':'Country'}, inplace=True)

function for replacing all the missings in the state column

In [9]:
def missings(state, country):
  return country if pd.isna(state) == True else state

In [10]:
train_df['State'] = train_df.apply(lambda x: missings(x['State'], x['Country']), axis=1)
test_df['State'] = test_df.apply(lambda x: missings(x['State'], x['Country']), axis=1)

In [11]:
display(train_df.isnull().sum() / len(train_df) * 100)
display(test_df.isnull().sum() / len(test_df) * 100)

Id                0.0
State             0.0
Country           0.0
Date              0.0
ConfirmedCases    0.0
Fatalities        0.0
dtype: float64

ForecastId    0.0
State         0.0
Country       0.0
Date          0.0
dtype: float64

In [12]:
print('In our dataset are {} countries and {} states'.format(train_df['Country'].nunique(), train_df['State'].nunique()))

In our dataset are 184 countries and 312 states


In [16]:
df_confirmedcases = train_df.groupby(['Country','State']).max().groupby('Country').sum().sort_values(by='ConfirmedCases', ascending=False).reset_index().drop(columns='Id')
df_confirmedcases

,Country,ConfirmedCases,Fatalities
0,US,1442653.0,87525.0
1,Russia,262843.0,2418.0
2,United Kingdom,238005.0,34078.0
3,Spain,230183.0,27459.0
4,Italy,223885.0,31610.0
...,...,...,...
179,Seychelles,11.0,0.0
180,Suriname,10.0,1.0
181,MS Zaandam,9.0,2.0
182,Papua New Guinea,8.0,0.0


In [17]:
df_confirmedcases[:20].set_index('Country').style.background_gradient(cmap='Oranges')

,ConfirmedCases,Fatalities
Country,,
US,1442653.000000,87525.000000
Russia,262843.000000,2418.000000
United Kingdom,238005.000000,34078.000000
Spain,230183.000000,27459.000000
Italy,223885.000000,31610.000000
Brazil,220291.000000,14962.000000
France,179630.000000,27532.000000
Germany,175233.000000,7897.000000
Turkey,146457.000000,4055.000000
